In [61]:
from news_leadgen.news_handler import NewsHandler
from news_leadgen.dto.newsdto import NewsDTO 
import pandas as pd

# Чтение вопросов к новостям

In [62]:
with open("questions.txt", "r") as f:
    txt =f.read()
questions = txt.split('\n')

# Чтение новостей из файла

In [63]:
news_data = pd.read_excel("Отказы MediaAnalysis_text 10М2023.xlsx", sheet_name=0)[["index", "text"]]
data = news_data.values

all_news = list()
for news_arr in data:
    all_news.append(NewsDTO(news_id=news_arr[0], text=news_arr[1]))

# Обработка новостей

In [64]:
nh = NewsHandler(questions = questions)

In [65]:
print(nh._get_questions_as_one_prompt())

Тебе дан список вопросов и одна новость. Ответь на вопросы, при подготовке ответа не фантазируй. 
        Если не знаешь ответа, так и скажи.
        Вопросы:0) Содержит ли текст новости информацию о строительстве, модернизации или расширении какого-либо промышленного, жилого, коммерческого объекта (или его части) или реализации инфраструктурного, инновационного проекта, который банк может профинансировать, ответь только ДА или НЕТ
1) Если текст новости содержит информацию о строительстве, модернизации или расширении какого-либо промышленного, жилого, коммерческого объекта (или его части) или реализации инфраструктурного, инновационного проекта, который банк может профинансировать, напиши краткое описание новости на 400 символов.
2) Напиши название компании о которой идёт речь в новости
3) Напиши регион о котором идёт речь в новости
4) Напиши ФИО действующих лиц о которых идёт речь в новости
5) Напиши тип инвестиционного проекта: промышленный, жилой, коммерческий, инфраструктурный, инн

## По одной

In [74]:
# возвращается сет: объект с результатом разбора (айди новости, ответ сети) и список заданных сети вопросов

one_news_processing_result, _ = nh.process_news(all_news[1])

ValidationError: 1 validation error for Messages
content
  str type expected (type=type_error.str)

In [25]:
print(one_news_processing_result.processing_result[0])

1. Главная проблема в Нижнем Тагиле — отток молодежи. Для решения этой проблемы необходимо сделать город более привлекательным для молодежи, предоставить им возможности для получения образования, стабильной работы с хорошим заработком, а также обеспечить доступность яслей и детских садов, школ.
2. Рекордный бюджет города позволит реализовать множество проектов по благоустройству города, развитию туризма, созданию комфортных условий для жителей.
3. Успевать все это удается благодаря опыту и профессионализму руководства города, а также тесному взаимодействию с жителями и поддержке местных предприятий.


## пачкой

In [10]:
#выделяю часть новостей чтобы не ждать обрабутку всех
news_batch = all_news[:3]

In [11]:
#результат - массив, каждый элемент которого это сет
#Состав сета: объект с резултьатом разбора (айди новости, ответ сети) и список заданных сети вопросов
news_processing_batch = nh.batch_process_news(news_batch)

In [12]:
npb_list = list()
for npr in news_processing_batch:
    npb_list.append((npr[1], npr[0].news_id, npr[0].processing_result))

In [26]:
responses = pd.DataFrame(data=npb_list, columns=["Заданные вопросы", "ID новости", "Ответы на вопросы"])
responses.head()

,Заданные вопросы,ID новости,Ответы на вопросы
0,0) Содержит ли текст новости информацию о стро...,ef2730c48257bbee6e68354038acd1ee,[1. Реконструкция реки Мостовой включает в себ...
1,0) Содержит ли текст новости информацию о стро...,e08e3641050bdabf89c99c518033ff93,[1. Главная проблема в Нижнем Тагиле — отток м...
2,0) Содержит ли текст новости информацию о стро...,fa2dd0fe1b710cd458ec28ce8e46e3b4,[1. Градостроительно-земельная комиссия (ГЗК) ...


In [28]:
responses.to_excel("responses.xlsx")